In [42]:
'''
Standard base sequence:
I(nsertion)0
P0: C:SYN
I1 
P1: S:SYNACK
I2
P2: C:ACK
I3
P3: C:request
'''


probes = []


sequence_level_strategy = [
    'Mutate',
    'Fragment',
    'Insert',
    'Delay'
]

import itertools
tcpFlags = ["S", "A", "R", "F", "P", "U"]

TCP_flag_combination = [
    '',
    'S',
    'A',
    'R',
    'F',
    'P',
    'SA',
    'SR',
    'SF',
    'SP',
    'RA',
    'FA',
    'PA',
    'FP',
    'RP',
    'PU',
    'PAU',
    'SAFPU',
    'SARFPU'
]

IP_flag_combination = [
    '',
    'E',
    'D',
    'M',
    'DM'
]

for seq_strategy in sequence_level_strategy:

    
    # 142 Mutate strategies
    if seq_strategy == 'Mutate':
        for mutate_layer in ['IP', 'TCP', 'Application']:
            if mutate_layer == 'IP':
                for mutate_field in ['flag', 'option', 'proto']:
                    if mutate_field == 'flag':
                        for mutate_ip_flag in IP_flag_combination:
                            cur_mutate_strategy = f'Mutate[layer:{mutate_layer};field:{mutate_field};flags:{mutate_ip_flag}]'
                            probes.append(cur_mutate_strategy)
                    elif mutate_field == 'option':
                        for mutate_ip_option in ['noop', 'recordRoute', 'invalid']:
                            cur_mutate_strategy = f'Mutate[layer:{mutate_layer};field:{mutate_field};option:{mutate_ip_option}]'
                            probes.append(cur_mutate_strategy)
                    elif mutate_field == 'proto':
                        for mutate_ip_proto in ['udp', 'invalid']:
                            cur_mutate_strategy = f'Mutate[layer:{mutate_layer};field:{mutate_field};proto:{mutate_ip_proto}]'
                            probes.append(cur_mutate_strategy)
            
            elif mutate_layer == 'TCP':
                for mutate_field in ['flag', 'option', 'urgentPointer', 'checksum', 'seqack']:
                    if mutate_field == 'flag':
                        for mutate_tcp_flag in TCP_flag_combination:
                            if mutate_tcp_flag == 'PA':
                                continue
                            cur_mutate_strategy = f'Mutate[layer:{mutate_layer};field:{mutate_field};flags:{mutate_tcp_flag}]'
                            probes.append(cur_mutate_strategy)
                    elif mutate_field == 'option':
                        for mutate_tcp_option in ['md5', 'timestamp', 'windowscale', 'fastopen', 'invalid']:
                            cur_mutate_strategy = f'Mutate[layer:{mutate_layer};field:{mutate_field};option:{mutate_tcp_option}]'
                            probes.append(cur_mutate_strategy)
                    elif mutate_field == 'urgentPointer':
                        for mutate_urgent_option in ['', 'nourg', 'noack', 'invalid']:
                            cur_mutate_strategy = f'Mutate[layer:{mutate_layer};field:{mutate_field};urgent:{mutate_urgent_option}]'
                            probes.append(cur_mutate_strategy)
                    elif mutate_field == 'checksum':
                        cur_mutate_strategy = f'Mutate[layer:{mutate_layer};field:{mutate_field};checksum:corrupt]'
                        probes.append(cur_mutate_strategy)
                    elif mutate_field == 'seqack':
                        for mutate_seqack_option in ['inwindowSeq', 'outwindowSeq', 'isn', 'outwindowSeqWithPadding', 'invalidAck']:
                            cur_mutate_strategy = f'Mutate[layer:{mutate_layer};field:{mutate_field};seqack:{mutate_seqack_option}]'
                            probes.append(cur_mutate_strategy)

            elif mutate_layer == 'Application':
                for mutate_application_type in ['domain', 'http', 'tls']:
                    if mutate_application_type == 'domain':
                        for mutate_domain_type in ['case', 'prepend', 'append']:
                            if mutate_domain_type == 'case':
                                for mutate_domain_case in ['upper', 'random']:
                                    cur_mutate_strategy = f'Mutate[layer:{mutate_layer};type:{mutate_application_type};change:{mutate_domain_type};case:{mutate_domain_case}]'
                                    probes.append(cur_mutate_strategy)
                            elif mutate_domain_type == 'prepend' or mutate_domain_type == 'append':
                                for mutate_domain_char in ['star', 'space']:
                                    cur_mutate_strategy = f'Mutate[layer:{mutate_layer};type:{mutate_application_type};change:{mutate_domain_type};char:{mutate_domain_char}]'
                                    probes.append(cur_mutate_strategy)
                    elif mutate_application_type == 'tls':
                        for mutate_tls_type in ['longCH', 'altVersion', 'insertRecord']:
                            if mutate_tls_type == 'longCH':
                                cur_mutate_strategy = f'Mutate[layer:{mutate_layer};type:{mutate_application_type};change:{mutate_tls_type}]'
                                probes.append(cur_mutate_strategy)
                            elif mutate_tls_type == 'altVersion':
                                for mutate_tls_version_position in ['recordVersion', 'legacyVersion']:
                                    for mutate_tls_version_value in ['0000', '0300', '0304', '03ff', 'ffff']:
                                        cur_mutate_strategy = f'Mutate[layer:{mutate_layer};type:{mutate_application_type};change:{mutate_tls_type};position:{mutate_tls_version_position};version:{mutate_tls_version_value}]'
                                        probes.append(cur_mutate_strategy)
                            elif mutate_tls_type == 'insertRecord':
                                for mutate_tls_recordType in ['helloRequest', 'emptyHandshake', 'emptyRecord', 'alertWarning']:
                                    cur_mutate_strategy = f'Mutate[layer:{mutate_layer};type:{mutate_application_type};change:{mutate_tls_type};recordType:{mutate_tls_recordType}]'
                                    probes.append(cur_mutate_strategy)
                    elif mutate_application_type == 'http':
                        for mutate_http_type in ['longRequest', 'insertChar', 'removeChar', 'altDelimiter', 'altMethod', 'altHostHeader', 'altVersion']:
                            if mutate_http_type == 'longRequest':
                                cur_mutate_strategy = f'Mutate[layer:{mutate_layer};type:{mutate_application_type};change:longRequest]'
                                probes.append(cur_mutate_strategy)
                                cur_mutate_strategy = f'Mutate[layer:{mutate_layer};type:{mutate_application_type};change:longlongRequest]'
                                probes.append(cur_mutate_strategy)
                            elif mutate_http_type == 'insertChar':
                                for mutate_http_insert_char in ['space', '?', 'tab']:
                                    for mutate_http_insert_num in ['1', '5']:
                                        for mutate_http_insert_position in ['1', '2', '3', '4', '5', '6']:
                                            cur_mutate_strategy = f'Mutate[layer:{mutate_layer};type:{mutate_application_type};change:{mutate_http_type};char:{mutate_http_insert_char};position:{mutate_http_insert_position};num:{mutate_http_insert_num}]'
                                            probes.append(cur_mutate_strategy)
                            elif mutate_http_type == 'removeChar':
                                for mutate_http_remove_position in ['1', '2', '3', '4', '5', '6']:
                                    cur_mutate_strategy = f'Mutate[layer:{mutate_layer};type:{mutate_application_type};change:{mutate_http_type};char:space;position:{mutate_http_remove_position}]'
                                    probes.append(cur_mutate_strategy)
                            elif mutate_http_type == 'altDelimiter':
                                for mutate_http_delimiter_char in ['r', 'n', '09', '0b', '0c', 'multiSpace']:
                                    cur_mutate_strategy = f'Mutate[layer:{mutate_layer};type:{mutate_application_type};change:{mutate_http_type};char:{mutate_http_delimiter_char}]'
                                    probes.append(cur_mutate_strategy)
                            elif mutate_http_type == 'altMethod':
                                for mutate_http_method in ['POST', 'PUT', 'GE', 'GeT']:
                                    cur_mutate_strategy = f'Mutate[layer:{mutate_layer};type:{mutate_application_type};change:{mutate_http_type};method:{mutate_http_method}]'
                                    probes.append(cur_mutate_strategy)
                            elif mutate_http_type == 'altHostHeader':
                                for mutate_http_hostheader in ['Hostheader', 'XXXX', 'Host']:
                                    cur_mutate_strategy = f'Mutate[layer:{mutate_layer};type:{mutate_application_type};change:{mutate_http_type};hostheader:{mutate_http_hostheader}]'
                                    probes.append(cur_mutate_strategy)
                            elif mutate_http_type == 'altVersion':
                                for mutate_http_version in ['HTTP/ 1.1', 'HtTP/1.1', 'HTTP/3']:
                                    cur_mutate_strategy = f'Mutate[layer:{mutate_layer};type:{mutate_application_type};change:{mutate_http_type};version:{mutate_http_version}]'
                                    probes.append(cur_mutate_strategy)
            
                        
    # 95 Fragment Strategies
    if seq_strategy == 'Fragment':

        for fragment_layer in ['IP', 'TCP', 'TLS']:
            for fragment_strategy in ['splitDomain', 'outorder', 'fragmentSize', 'fragmentNum', 'overlap', 'timeout', 'maxDist', 'tcpEncapsulation']:
                if fragment_strategy == 'splitDomain':
                    cur_fragment_strategy = f'Fragment[layer:{fragment_layer};type:{fragment_strategy}]'
                    #probes.append(cur_fragment_strategy)
                elif fragment_strategy == 'outorder':
                    cur_fragment_strategy = f'Fragment[layer:{fragment_layer};type:{fragment_strategy}]'
                    probes.append(cur_fragment_strategy)
                elif fragment_strategy == 'tcpEncapsulation':
                    if fragment_layer != 'TLS':
                        continue
                    cur_fragment_strategy = f'Fragment[layer:{fragment_layer};type:{fragment_strategy}]'
                    probes.append(cur_fragment_strategy)
                elif fragment_strategy == 'fragmentSize':
                    for fragment_size in ['8', '16', '24', '32', '40', '48', '56', '64']:
                        cur_fragment_strategy = f'Fragment[layer:{fragment_layer};type:{fragment_strategy};size:{fragment_size}]'
                        probes.append(cur_fragment_strategy)
                elif fragment_strategy == 'fragmentNum':
                    for fragment_num in ['2', '3', '4', '8', '16', '32', '55']:
                        cur_fragment_strategy = f'Fragment[layer:{fragment_layer};type:{fragment_strategy};num:{fragment_num}]'
                        if fragment_layer == 'TLS' and int(fragment_num) > 16:
                            continue
                        probes.append(cur_fragment_strategy)
                elif fragment_strategy == 'timeout':
                    if fragment_layer == 'TLS':
                        continue
                    for fragment_timeout in ['5', '10', '15', '30', '45']:
                        cur_fragment_strategy = f'Fragment[layer:{fragment_layer};type:{fragment_strategy};timeout:{fragment_timeout}]'
                        probes.append(cur_fragment_strategy)
                elif fragment_strategy == 'maxDist':
                    if fragment_layer != 'IP':
                        continue
                    for fragment_maxDist in ['16', '32', '64']:
                        cur_fragment_strategy = f'Fragment[layer:{fragment_layer};type:{fragment_strategy};maxdist:{fragment_maxDist}]'
                        probes.append(cur_fragment_strategy)
                elif fragment_strategy == 'overlap':
                    if fragment_layer == 'TLS':
                        continue
                    for fragment_overlap in ['lshort_rshort', 'lshort_requal', 'lshort_rlong',
                                             'lequal_rshort', 'lequal_requal', 'lequal_rlong',
                                             'llong_rshort', 'llong_requal', 'llong_rlong', 'lequal_requal_outorder']:
                        cur_fragment_strategy = f'Fragment[layer:{fragment_layer};type:{fragment_strategy};overlap:{fragment_overlap}]'
                        #probes.append(cur_fragment_strategy)
                
    
    
    elif seq_strategy == 'Insert':

        # 7296 insert strategies
        for insert_position in ['I0', 'I1', 'I2', 'I3']:
            for insert_flag in TCP_flag_combination:
                for insert_data in ['', 'random', 'controlRequest', 'altProto']:
                    for insert_option in ['', 'md5', 'timestamp', 'corruptChecksum', 'inwindowSeq', 'outwindowSeq', 'isn', 'wrongACK']:
                        for insert_repeat in ['0', '1', '2']:
                            cur_insert_strategy = f'Insert[position:{insert_position};flag:{insert_flag};data:{insert_data};option:{insert_option};repeat:{insert_repeat}]'
                            #probes.append(cur_insert_strategy)

    # 10 delay strategies
    elif seq_strategy == 'Delay':
        for delay_position in ['I1', 'I3']:
            for delay_time in ['5', '10', '15', '30', '45']:
                cur_delay_strategy = f'Delay[position:{delay_position};time:{delay_time}]'
                #probes.append(cur_delay_strategy)


len(probes)

184

In [48]:
template = r'''
protocol: both
message:
  http: 
    request: "GET / HTTP/1.1\r\nHost: ${}\r\nUser-Agent: curl/8.11.1\r\nAccept: */*\r\n\r\n"
  tls:
    clientHelloConfig:
      chVersion: "0303"
    records:
      - contentType: "16"
        recordVersion: "0301"
        payloadType: "clienthello"
        offset: 0
        length: -1

packets:
  - ethernet:
    ip:
      tos: 0
      ttl: 64
      id: 33345
    tcp:
      window: 65535
      flags:
        syn: true
      tcpOptions:
        - tcpOptionType: 1
        - tcpOptionType: 1
        - tcpOptionType: 2
          tcpOptionLength: 4
          tcpOptionData: "05B4"
        - tcpOptionType: 3
          tcpOptionLength: 3
          tcpOptionData: "06"
    delay: 1

  - ethernet:
    ip:
      tos: 0
      ttl: 64
      id: 0
    tcp:
      window: 2056
      flags:
        syn: true
        ack: true
      tcpOptions:
        - tcpOptionType: 1
        - tcpOptionType: 1
        - tcpOptionType: 2
          tcpOptionLength: 4
          tcpOptionData: "05B4"
        - tcpOptionType: 3
          tcpOptionLength: 3
          tcpOptionData: "06"
    serverPacket: true
    delay: 1
      
  - ethernet:
    ip:
      tos: 0
      ttl: 64
      id: 33346
    tcp:
      window: 2056
      flags:
        ack: true
    delay: 1

  - ethernet:
    ip:
      tos: 0
      ttl: 64
      id: 33347
    tcp:
      window: 2056
      flags:
        psh: true
        ack: true
      messageOffset: 0
      messageLength: -1
    delay: 1
'''

for i in range(183, len(probes)):
    cur_file = open('probes/' + str(i+1) + '.yml', 'w')
    cur_file.write('# yaml-language-server: $schema=./../cendpi-schema.json\n')
    cur_file.write("strategy: " + probes[i])
    cur_file.write(template)
    cur_file.close()



In [46]:
probes[182]

'Fragment[layer:TLS;type:fragmentNum;num:16]'

In [10]:
probes[57]

'Mutate[layer:Application;type:http;change:insertChar;char:space;position:1;num:5]'

In [39]:
def print_yaml_blocks(n):
    for i in range(n):
        print("      - contentType: \"16\"")
        print("        recordVersion: \"0301\"")
        print("        payloadType: \"clienthello\"")
        print(f"        offset: {i * 16}")
        print("        length: 16")

print_yaml_blocks(16)

      - contentType: "16"
        recordVersion: "0301"
        payloadType: "clienthello"
        offset: 0
        length: 16
      - contentType: "16"
        recordVersion: "0301"
        payloadType: "clienthello"
        offset: 16
        length: 16
      - contentType: "16"
        recordVersion: "0301"
        payloadType: "clienthello"
        offset: 32
        length: 16
      - contentType: "16"
        recordVersion: "0301"
        payloadType: "clienthello"
        offset: 48
        length: 16
      - contentType: "16"
        recordVersion: "0301"
        payloadType: "clienthello"
        offset: 64
        length: 16
      - contentType: "16"
        recordVersion: "0301"
        payloadType: "clienthello"
        offset: 80
        length: 16
      - contentType: "16"
        recordVersion: "0301"
        payloadType: "clienthello"
        offset: 96
        length: 16
      - contentType: "16"
        recordVersion: "0301"
        payloadType: "clienthello"
       